# Stock Prediction
### Ignacio Fernández Adrados

---

In [ ]:
! pip install numpy 
! pip install pandas
! pip install scikit-learn
! pip install matplotlib
! pip install numpy
! pip install keras
! pip install tensorflow
! pip install torch


In [ ]:
import pandas as pd # type: ignore

def getStockDataFrame (stock, years):
    STOCK = stock
    YEARS = years

    file = None
    for year in YEARS:
        FILE='Datasets/Stock-'+STOCK+'-'+year+'.csv'
        ds=pd.read_csv(FILE, sep=',')
        file = pd.concat([file,ds])
    file = file.sort_values(by='timestamp', ascending=True)
    return file
#getStockDataFrame('AAPL', ['2022','2023'])

In [ ]:
from datetime import datetime

def groupStocks(dataset, n, datefilter):

    N=n     #Minutos agrupados
    DATEFILTER=datefilter #Filtro de fecha
    file = dataset
    
    df = file[file['timestamp'].str.startswith(DATEFILTER)]
    df=pd.DataFrame(df)
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    dfgroup = pd.DataFrame(df.groupby(pd.Grouper(key='timestamp', freq=N)).first()['open'])
    dfgroup = dfgroup.join(df.groupby(pd.Grouper(key='timestamp', freq=N)).last()['close'])
    dfgroup = dfgroup.join(df.groupby(pd.Grouper(key='timestamp', freq=N)).min()['low'])
    dfgroup = dfgroup.join(df.groupby(pd.Grouper(key='timestamp', freq=N)).max()['high'])
    dfgroup = dfgroup.join(df.groupby(pd.Grouper(key='timestamp', freq=N)).sum()['volume'])
    #dfgroup = dfgroup[:-1].reset_index(drop=True)
    dfgroup = pd.DataFrame(pd.to_datetime(dfgroup.index[:])).join(dfgroup.reset_index(drop=True))
    dfgroup = dfgroup.dropna().reset_index(drop=True)
    return dfgroup
#xxx = groupStocks(getStockDataFrame('AAPL', ['2022']), '1D', '')
#xxx

In [ ]:

def get_rsi(close, lookback):
    ret = close.diff()
    up = []
    down = []
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()
    rs = up_ewm/down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df[3:]

def getIndicators(dataset):
    
    dfgroup = dataset

    dfgroup['EMA7']= dfgroup['close'].ewm(span=7, adjust=False).mean()
    dfgroup['MACD']= dfgroup['close'].ewm(span=12, adjust=False).mean()- dfgroup['close'].ewm(span=26, adjust=False).mean()
    dfgroup['SignalMACD'] = dfgroup['MACD'].ewm(span=9, adjust=False).mean()
    dfgroup['RSI'] = get_rsi(dfgroup['close'], 14)
    dfgroup = dfgroup.dropna()
    dfgroup = dfgroup.reset_index(drop=True)
    return dfgroup

#getIndicators(xxx)

Añadiendo sentimiento

In [118]:
def getNewsDataFrame(stock, dates, interval, stockDataFrame, relevance):
   
    file = None
    for year in dates:
        FILE='Datasets/News-'+stock+'-'+year+'.csv'
        ds=pd.read_csv(FILE, sep=',')
        file = pd.concat([file,ds])
    file = file.sort_values(by='date', ascending=True)
    
    file['date'] = pd.to_datetime(file['date'], format='%Y%m%dT%H%M%S')
    file = file.drop(['title','summary','ticker'], axis=1)
    file = file[file['relevance']>=relevance]
    
    dfgroup = pd.DataFrame(file.groupby(pd.Grouper(key='date', freq=interval)).last()['relevance'])
    dfgroup = dfgroup.join(file.groupby(pd.Grouper(key='date', freq=interval)).last()['sentiment'])
    dfgroup = pd.DataFrame(pd.to_datetime(dfgroup.index[:])).join(dfgroup.reset_index(drop=True))
    dfgroup = dfgroup.fillna(0)
    stockDataFrame['timestamp'] = pd.to_datetime(stockDataFrame['timestamp'])
    res = stockDataFrame.join(dfgroup.set_index("date"), on='timestamp', how='left')
    res = res.fillna(0)
    res = res.reset_index(drop=True)

    return res

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalize(dataset):
    #Normalise data into (0,1) range
    normData=dataset
    scaler = MinMaxScaler(feature_range=(0, 1))

    #normData[['open','close','low','high','volume','EMA7','MACD','SignalMACD','RSI']] = \
    #    scaler.fit_transform(normData[['open','close','low','high','volume','EMA7','MACD','SignalMACD','RSI']])
    #print(normData[['open','close','low','high','volume','EMA7','MACD','SignalMACD','RSI']].head(1))
    c=normData.columns.values
    normData[c] = scaler.fit_transform(normData[c])
    return normData

##################################################3
#data = getStockDataFrame('AAPL', ['2023'])
#indicatorData = groupStocks(data, '1D', '')
#indicatorData = getNewsDataFrame('AAPL', ['2023'], '1D', indicatorData, 0)
#indicatorData = indicatorData.drop(['timestamp'], axis=1)
#normalize(indicatorData)
##################################################3

In [ ]:
import matplotlib.pyplot as plt

def plotDataset(dataset):

    normData = dataset
    
    plt.figure(figsize=(14,4))
    plt.title("Dataset 'Close' data ")
    plt.plot(normData['close'],color='black')
    #plt.scatter(normData.index, np.where(normData['buysell'] == -1,normData['close'], None), color="red", marker="v")
    #plt.scatter(normData.index, np.where(normData['buysell'] ==  1,normData['close'], None), color="blue", marker="^")
    plt.show()

### Split the values in train and test

So, we took only 25% of the data as training samples and set aside the rest of the data for testing.

Looking at the time-series plot, we think **it is not easy for a standard model to come up with correct trend predictions.**

In [ ]:
def splitData(dataset, split, step):

    S=split
    step = step
    normData = dataset

    split = int(len(normData) * S)
    #values = normData.values
    #print(values)
    train = normData[:split]#.drop(['buysell'],axis=1)
    test = pd.concat([train.tail(step),normData[split:]]).reset_index(drop=True)

    #print("Train data length:", train.shape)
    #print("Test data length:", test.shape)

    return train, test

In [ ]:
def plotSplitDataset(dataset, split):
    
    normData = dataset
    split = int(len(normData) * split)

    plt.figure(figsize=(14,4))
    plt.title("Dataset 'Close' data split")
    plt.plot(normData.index.values,normData['close'],c='black')
    plt.axvline(normData.index[split], c="r")
    #plt.scatter(normData.index, np.where(normData['buysell'] == -1,normData['close'], None), color="red",  marker="v")
    #plt.scatter(normData.index, np.where(normData['buysell'] ==  1,normData['close'], None), color="blue", marker="^")
    plt.show()


### Converting to a multi-dimensional array
Next, we'll convert test and train data into the matrix with step value as it has shown above example.

In [ ]:
import numpy as np

def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step
        #print(i, d, data[i:d)
        X.append(data[i:d])
        Y.append(data[d,1])
    return np.array(X), np.array(Y)

def dataset2Matrix(train, test, step):
    trainX,trainY =convertToMatrix(train.to_numpy(),step)
    testX,testY =convertToMatrix(test.to_numpy(),step)
    #print(trainY)
    #print("Training data shape:", trainX.shape,', ',trainY.shape)
    #print("Test data shape:", testX.shape,', ',testY.shape)
    return trainX, trainY, testX, testY


### Keras model with `SimpleRNN` layer

- 256 neurons in the RNN layer
- 32 denurons in the densely connected layer
- a single neuron for the output layer
- ReLu activation
- learning rate: 0.001

In [ ]:
from keras import Sequential
from keras.layers import Dense, SimpleRNN, Input, LSTM, Dropout # type: ignore

def createModel(type, units, trainX, step):
    UNITS = units #num_units: Number of units of a the simple RNN layer
    DENSEUNITS = 32 #Number of neurons in the dense layer followed by the RNN layer
    
    model = Sequential()
    model.add(Input((step, trainX.shape[2])))
    if type=="LSTM":
        model.add(LSTM(units=UNITS, activation="tanh",return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(units=UNITS//2, activation="tanh"))
        model.add(Dropout(0.2))      
    elif type == "RNN":
        model.add(SimpleRNN(units=UNITS, activation="relu",return_sequences=True))
        model.add(Dropout(0.2))
        model.add(SimpleRNN(units=UNITS//2, activation="relu"))
        model.add(Dropout(0.2))     
    model.add(Dense(DENSEUNITS, activation="tanh"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mse'])    
    return model

### Fit the model

In [ ]:
from keras.callbacks import Callback # type: ignore

class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        #if (epoch+1) % 10 == 0 and epoch>0:
            print("Epoch number {} done".format(epoch+1))

def trainModel(model, batch, epochs, trainX, trainY):
    batch_size=batch
    num_epochs = epochs

    model.fit(trainX,trainY,
            epochs=num_epochs,
            batch_size=batch_size,
            callbacks=None,verbose=0
            )
    return model

### Plot loss

In [ ]:
def plotLosss(model):
    plt.figure(figsize=(8,3))
    plt.title("RMSE loss over epochs",fontsize=16)
    plt.plot(np.sqrt(model.history.history['loss']),c='k',lw=2)
    plt.grid(True)
    plt.xlabel("Epochs",fontsize=14)
    plt.ylabel("Root-mean-squared error",fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()

### Predictions
Note that the model was fitted only with the `trainX` and `trainY` data.

In [122]:
def predict(model, trainX, testX):
    trainPredict = model.predict(trainX, verbose=0)
    testPredict= model.predict(testX, verbose=0)
 
    #predicted=np.concatenate((trainPredict,testPredict),axis=0)
    #print(trainPredict.shape)
    #print(testPredict.shape)

    return trainPredict, testPredict


### Comparing it with the ground truth (test set)

In [ ]:
def plotCompare(normData, testPredict, split):

    OFFSET=0
    split = int(len(normData) * split)
    #index = normData.index.values
    plt.figure(figsize=(14,4))
    plt.title("Ground truth and prediction together",fontsize=18)
    plt.plot(normData['close'].iloc[split+OFFSET:].reset_index(drop=True),color='black', label='Ground Truth')
    plt.plot(testPredict[OFFSET:],color='blue', label='Prediction')
    plt.legend()
    plt.show()

In [ ]:
def calculateDecision(test, testPredict, step, profit):

    PROFIT = profit

    decision=test['close'].iloc[step:]
    decision = decision.diff()
    decision = decision.dropna()
    decision = np.where(abs(decision)<PROFIT,0,np.sign(decision).astype('int'))
    decision = pd.DataFrame(data={'buysell':decision})#.drop(0).reset_index(drop=True)

    predictedDecision = pd.DataFrame(data={'buysellPredicted':testPredict[:,0]})
    predictedDecision = predictedDecision.diff()
    predictedDecision = predictedDecision.dropna()#.reset_index(drop=True)
    predictedDecision = np.where(abs(predictedDecision)<PROFIT,0,np.sign(predictedDecision).astype('int'))
    predictedDecision = pd.DataFrame(data={'buysellPredicted':predictedDecision[:,0]})

    decision = decision.join(predictedDecision)
    return decision

In [ ]:
def plotResult(normData, testPredict, split, profit, decision):

    PROFIT=profit
    OFFSET=0
    split = int(len(normData) * split)    

    index = normData.index.values
    plt.figure(figsize=(14,4))
    plt.title("Ground truth and prediction together",fontsize=18)
    plt.plot(normData['close'].iloc[split+OFFSET:].reset_index(drop=True),color='black')
    plt.plot(testPredict[OFFSET:],color='blue')

    x = decision.iloc[OFFSET:].reset_index(drop=True).index
    y = normData['close'].iloc[len(normData)-len(decision.index)-1+OFFSET:-1].reset_index(drop=True)

    plt.scatter(x, np.where(decision['buysell'].iloc[OFFSET:] == -1,y, None), color="red",  marker="v", label='Ground Truth Sell')
    plt.scatter(x, np.where(decision['buysell'].iloc[OFFSET:] ==  1,y, None), color="blue", marker="^", label='Ground Truth Buy')

    plt.scatter(x, np.where(decision['buysellPredicted'].iloc[OFFSET:] == -1,y-0.015, None), color="orange",  marker="v", label='Prediction Sell')
    plt.scatter(x, np.where(decision['buysellPredicted'].iloc[OFFSET:] ==  1,y+0.015, None), color="green",  marker="^", label='Prediction Buy')
    plt.legend()
    plt.show()


In [ ]:
def plotProfit(normData, split, decision):

    OFFSET=0
    split = int(len(normData) * split)
    index = normData.index.values
    plt.figure(figsize=(14,4))
    plt.title("Ground truth and prediction together",fontsize=18)
    plt.plot(normData['close'].iloc[split+OFFSET:].reset_index(drop=True),color='black')
    #plt.plot(testPredict[OFFSET:],color='blue')

    x = decision.iloc[OFFSET:].reset_index(drop=True).index
    y = normData['close'].iloc[len(normData)-len(decision.index)-1+OFFSET:-1].reset_index(drop=True)

    plt.scatter(x, np.where((decision['buysell'].iloc[OFFSET:] !=  decision['buysellPredicted'].iloc[OFFSET:]) & (decision['buysellPredicted'].iloc[OFFSET:] != 0),y, None), color="red", label='Ground Truth Buy')
    plt.scatter(x, np.where((decision['buysell'].iloc[OFFSET:] == decision['buysellPredicted'].iloc[OFFSET:]) & (decision['buysell'].iloc[OFFSET:] != 0),y, None), color="green",  label='Ground Truth Sell')

    plt.legend()
    plt.show()

In [ ]:
def calculateProfit(normData, decision):

    OFFSET=0
    y = normData['close'].iloc[len(normData)-len(decision.index)-1+OFFSET:-1].reset_index(drop=True)
    
    #print("Predicciones no coincidentes con el conjunto de pruebas: ", \
    nocoincide =np.count_nonzero(np.where((decision['buysell'].iloc[OFFSET:] !=  decision['buysellPredicted'].iloc[OFFSET:]) \
          & (decision['buysellPredicted'].iloc[OFFSET:] != 0),y, None))#)
    #print("Predicciones coincidentes con el conjunto de pruebas: ", \
    coincide=np.count_nonzero(np.where((decision['buysell'].iloc[OFFSET:] == decision['buysellPredicted'].iloc[OFFSET:]) \
          & (decision['buysell'].iloc[OFFSET:] != 0),y, None))#)
    return coincide, nocoincide

In [117]:
def  getFinBERTdata(stock, dates):
    
    from transformers import BertTokenizer, BertForSequenceClassification, pipeline
    #import transformers

    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

    nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
    '''
    results = nlp(['growth is strong and we have plenty of liquidity.', 
               'there is a shortage of capital, and we need extra financing.',
              'formulation patents might protect Vasotec to a limited extent.'])
    print(results)
    '''
    file = None
    for year in dates:
        FILE='Datasets/News-'+stock+'-'+year+'.csv'
        ds=pd.read_csv(FILE, sep=',')
        file = pd.concat([file,ds])
    file = file.sort_values(by='date', ascending=True)
    file = file[file.relevance >0.7].reset_index(drop=True)

    headlines_array = np.array(file)
    headlines_list = list(headlines_array[:,1])
    
    results =nlp(headlines_list)
    #print(len(results))
    pdr = pd.DataFrame(results)
    #print(pdr)
    file["FinBertScore"]= pdr["score"]
    file["FinBertLabel"]= pdr["label"]
    return file

In [ ]:
def getFinbertNewsDataFrame(data, newsdata, interval):
    
    file = newsdata.sort_values(by='date', ascending=True)
    file['date'] = pd.to_datetime(file['date'], format='%Y%m%dT%H%M%S')
    file = file.drop(['title','summary','ticker', 'relevance','sentiment', 'FinBertLabel'], axis=1)
        
    dfgroup = pd.DataFrame(file.groupby(pd.Grouper(key='date', freq=interval)).last()['FinBertScore'])
    
    dfgroup = pd.DataFrame(pd.to_datetime(dfgroup.index[:])).join(dfgroup.reset_index(drop=True))
    dfgroup = dfgroup.fillna(0)
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    res = data.join(dfgroup.set_index("date"), on='timestamp', how='left')
    res = res.fillna(0)
    res = res.reset_index(drop=True)

    return res    

In [134]:
def runTest(split, stock, dates, profitMult, intervals, algorithms,  steps, \
            units, batchs, profits, epochs, sentiment, relevance, useIndicators, \
            useFinBERT, printFinalData=False, printSplitDataset=False, \
            printModelSummary=False, printLoss=False, printCompare=False, \
            printResult=False, printProfit=False):
    
    df = None
    numberoftests = len(intervals) * len(algorithms) * len(steps) * len(units) *  \
    len(batchs) * len(profits) * len(epochs) * len(sentiment) * len(useIndicators) * len(useFinBERT)
    
    if True in useFinBERT:
        print("Calculating FinBERT finantial sentiment predictions...")
        newsdata = getFinBERTdata(stock, dates)
        print("Finantial sentiment stored in dataset")
    
    print ('Running', numberoftests, 'tests:')
    
    for interval in intervals:
        for algorithm in algorithms:
            for step in steps:
                for unit in units:
                    for batch in batchs:
                        for epoch in epochs:
                            for indicator in useIndicators:
                                for sent in sentiment:
                                    for finbert in useFinBERT:

                                        data = getStockDataFrame(stock, dates )
                                        data = groupStocks(data,interval, '')
                                        
                                        if useIndicators:
                                            data = getIndicators(data)
                                        else:
                                            data = data                        
                                        
                                        if sent:
                                            data = getNewsDataFrame(stock, dates, \
                                                    interval, data, relevance)
                                          
                                        if finbert:
                                            data = getFinbertNewsDataFrame(data, newsdata, interval)
                                        data 
                                        data = data.drop(["timestamp"], axis=1)
                                        finalData = normalize(data)                            
                                     
                                        if printFinalData:
                                            plotDataset(finalData)
                                        print(finalData.shape)
                                        train, test = splitData(finalData, split, step)

                                        if printSplitDataset:
                                            plotSplitDataset(finalData, split)

                                        trainX, trainY, testX, testY = dataset2Matrix(train, test, step)
                                        model = createModel(algorithm, unit,trainX, step)
                                        
                                        if printModelSummary:
                                            model.summary()
                                        
                                        trainedModel = trainModel(model,batch,epoch,trainX,trainY)
                                        if printLoss:
                                            plotLosss(trainedModel)
                                        loss = np.sqrt(trainedModel.history.history['loss'][-1])
                                        print(testX.shape)                       
                                        trainPredict, testPredict = predict(trainedModel,trainX, testX)
                                        if printCompare:
                                            plotCompare(finalData, testPredict, split)

                                        for profit in profits:
                                            decision = calculateDecision(test, testPredict, step, profit*profitMult)
                                            
                                            if printResult:
                                                plotResult(finalData, testPredict,split, profit, decision)
                                            if printProfit:
                                                plotProfit(finalData, split, decision)
                                            coincide, nocoincide = calculateProfit(finalData, decision)
                                            if coincide+nocoincide == 0:
                                                ben = '-'
                                            else:
                                                ben = str(round(coincide/(coincide+nocoincide)*100,2))+'%'
                                            df = pd.concat([df, pd.DataFrame({
                                                'Interval': ["{:>3}".format(interval)],
                                                'Algorithm': ["{:>5}".format(algorithm)],
                                                'Steps':["{:3}".format(step)],
                                                'Units':["{:4}".format(unit)],
                                                'Batchs':["{:5}".format(batch)],
                                                'Epochs':["{:3}".format(epoch)],
                                                'ExpectedProfit':["{:11.8f}".format(profit*profitMult)],
                                                'RMSELoss': ["{:9.6f}".format(loss)],
                                                'Indicators' : ["{:>6}".format(str(indicator))],
                                                'CalculatedSentiment' : ["{:>6}".format(str(sent))],
                                                'FinBERTSentiment' : ["{:>6}".format(str(finbert))],
                                                'Relevance' : ["{:>4}".format(relevance)],
                                                'GoodPrediction': ["{:4}".format(coincide)],
                                                'BadPrediction': ["{:4}".format(nocoincide)],
                                                'Accuracy': ["{:>7}".format(ben)]                                    
                                            })])

                                            print(df.iloc[-1:].to_string(header=False, index=False))
                                            
                                            #Export dataset to CSV
                                            df.to_csv('Datasets/Tests_Results-' + stock + str(dates)+ \
                                                'Interval-' + str(intervals) + str(algorithms) + 'Steps-' + \
                                                str(steps) + 'Units-' + str(units) + 'Batchs-' + str(batchs)+ \
                                                'Epochs-' + str(epochs) + 'profitMult-' + str(profitMult) + \
                                                'Sentiment-' + str(sentiment) + 'FinBERT-' + str(useFinBERT) + \
                                                'Indicators' + str(useIndicators) + '.csv', index=False)                                
    df.reset_index(drop=True)
    print("Tests finished")


In [135]:
runTest(
    split = 0.7, 
    stock ='AAPL', 
    dates = ['2022','2023'],        #['2022','2023']
    profitMult = 0.000001,
    intervals = ['12h'],            #['1min', '10min', '60min', '6h', '12h', '1D']
    algorithms = ['LSTM'],          #['RNN', 'LSTM']
    steps = [10, 20],               #[2, 5, 10, 20]
    units = [100,150,200],                  #[50, 100., 150]
    batchs = [16,64,128],           #[16,64,128]
    profits = [0.1,1,10,100,1000] ,     #[1,10,100,1000]
    epochs = [5, 10],               #[5, 10, 20]
    sentiment = [True, False],            #[True, False]
    useFinBERT =[True, False],             #[True, False]
    relevance = 7.0,                # 0.0-1.0
    useIndicators = [True, False],  #[True, False]
    printFinalData=False, 
    printSplitDataset=False, 
    printModelSummary=False, 
    printLoss=False, 
    printCompare=False, 
    printResult=False, 
    printProfit=False
    ) 
#Arreglar títulos de los gráficos


Calculating FinBERT finantial sentiment predictions...
Finantial sentiment stored in dataset
Running 1440 tests:
(998, 12)
(300, 10, 12)
12h  LSTM  10  100    16   5  0.00000010  0.066171   True   True   True  7.0  174  125  58.19%
12h  LSTM  10  100    16   5  0.00000100  0.066171   True   True   True  7.0  174  125  58.19%
12h  LSTM  10  100    16   5  0.00001000  0.066171   True   True   True  7.0  174  124  58.39%
12h  LSTM  10  100    16   5  0.00010000  0.066171   True   True   True  7.0  174  122  58.78%
12h  LSTM  10  100    16   5  0.00100000  0.066171   True   True   True  7.0  156  115  57.56%
(998, 11)
(300, 10, 11)
12h  LSTM  10  100    16   5  0.00000010  0.064664   True   True  False  7.0  176  123  58.86%
12h  LSTM  10  100    16   5  0.00000100  0.064664   True   True  False  7.0  176  123  58.86%
12h  LSTM  10  100    16   5  0.00001000  0.064664   True   True  False  7.0  176  122  59.06%
12h  LSTM  10  100    16   5  0.00010000  0.064664   True   True  False  7.0  1

KeyboardInterrupt: 